**T5**

In [ ]:
pip install happytransformer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import csv

# JFLEG Dataset

train_dataset = load_dataset("jfleg", split='validation[:]')
eval_dataset = load_dataset("jfleg", split='test[:]')

print(train_dataset)
print(train_dataset['sentence'][0])
print(train_dataset['corrections'][0])

In [ ]:
replacements = [
  (" .", "."),
  (" ,", ","),
  (" '", "'"),
  (" ?", "?"),
  (" !", "!"),
  (" :", "!"),
  (" ;", "!"),
  (" n't", "n't"),
  (" v", "n't"),
  ("2 0 0 6", "2006"),
  ("5 5", "55"),
  ("4 0 0", "400"),
  ("1 7-5 0", "1750"),
  ("2 0 %", "20%"),
  ("5 0", "50"),
  ("1 2", "12"),
  ("1 0", "10"),
  ('" ballast water', '"ballast water')
]

def remove_excess_spaces(text):
  for rep in replacements:
    text = text.replace(rep[0], rep[1])

  return text

def generate_csv(csv_path, dataset):
    with open(csv_path, 'w', newline='') as csvfile:
        writter = csv.writer(csvfile)
        writter.writerow(["input", "target"])
        for case in dataset:
     	    # Adding the task's prefix to input
            input_text = "grammar: " + case["sentence"]
            input_text = remove_excess_spaces(input_text)
            for correction in case["corrections"]:
              correction = remove_excess_spaces(correction)
              # a few of the cases contain blank strings.
              if input_text and correction:
                writter.writerow([input_text, correction])

In [ ]:
# Generate train and eval for JFLEG Dataset
generate_csv("/content/drive/MyDrive/Kadir_TEZ/train.csv", train_dataset)
generate_csv("/content/drive/MyDrive/Kadir_TEZ/eval.csv", eval_dataset)

In [ ]:
import pandas as pd
train_df= pd.read_csv("/content/drive/MyDrive/Kadir_TEZ/train.csv")

In [ ]:
train_df

,input,target
0,grammar: So I think we can not live if old peo...,So I think we would not be alive if our ancest...
1,grammar: So I think we can not live if old peo...,So I think we could not live if older people d...
2,grammar: So I think we can not live if old peo...,So I think we can not live if old people could...
3,grammar: So I think we can not live if old peo...,So I think we can not live if old people can n...
4,grammar: For not use car.,Not for use with a car.
...,...,...
3011,grammar: Sceene ofn'tiolence can affect on them.,Scenes ofn'tiolence can affect them.
3012,grammar: While the communities in general have...,The communities in general have reckoned that ...
3013,grammar: While the communities in general have...,While the communities in general have reckoned...
3014,grammar: While the communities in general have...,While the communities in general have recogniz...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from happytransformer import HappyTextToText, TTTrainArgs

args = TTTrainArgs(batch_size=8)
happy_tt = HappyTextToText("T5", "t5-base")

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
before_result = happy_tt.eval("/content/drive/MyDrive/lang8/JFLEG/eval.csv", args=args)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-754c14227b98ae72/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
***** Running Evaluation *****
  Num examples = 2988
  Batch size = 8
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
print("Loss before training: ", before_result.loss)


Loss before training:  1.4125795364379883


In [ ]:
# Training the model
happy_tt.train("/content/drive/MyDrive/lang8/JFLEG/train.csv", args=args)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5f6d866dc7ceccef/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/3016 [00:00<?, ? examples/s]

PyTorch: setting up devices
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 3016
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1131
  Number of trainable parameters = 222903552


Step,Training Loss
500,0.621800
1000,0.480200




Training completed. Do not forget to share your model on huggingface.co/models =)




In [ ]:
after_loss = happy_tt.eval("/content/drive/MyDrive/lang8/JFLEG/eval.csv", args=args)
print("Loss after training: ", after_loss.loss)

In [ ]:
from happytransformer import TTSettings
beam_settings = TTSettings(num_beams=5, min_length=1, max_length=50)

In [ ]:
import csv
import json
import flask
from flask import jsonify

In [ ]:
import pickle

modelt5 = pickle.dump( open('/content/drive/MyDrive/Kadir_TEZ/model.pkl','rb'))

In [ ]:
beam_settings = TTSettings(num_beams=5, min_length=1, max_length=20)

model = pickle.load(open('/content/drive/MyDrive/Kadir_TEZ/model.pkl', 'rb'))
output = model.generate_text("I am new to learns english.", args=beam_settings)

In [ ]:
print(output)

TextToTextResult(text='to learns english. I am new to learns english.')


In [ ]:
with open('/content/drive/MyDrive/Kadir_TEZ/sentence_2000_grammer_error.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        sutun1 = "grammar: "+row[0]
        happy_tt.generate_text(sutun1, args=beam_settings)

In [ ]:
example_1 = "grammar: I am new to learns english."
result_1 = happy_tt.generate_text(example_1, args=beam_settings)
print(result_1)

TextToTextResult(text='False')


In [ ]:

example_2 = "grammar: Rajesh invite me yesterday on his birthday party."
result_2 = happy_tt.generate_text(example_2, args=beam_settings)
print(result_2.text)

False


**GPT-2**

**Attention biLSTM**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
import pandas as pd
import re
import datetime
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import nltk.translate.bleu_score as bleu
import tensorflow as tf
from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM, Flatten, Activation, Bidirectional, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import TensorBoard
import matplotlib.ticker as ticker
import warnings
warnings.filterwarnings("ignore")

In [ ]:
model = tf.saved_model.load('/content/drive/MyDrive/Kadir_TEZ/biLSTM/')

OSError: ignored

In [ ]:
def predict(input_sentence):

  batch_size = 1
  DECODER_SEQ_LEN = 25
  predict_word_idx = np.zeros((1, 1))
  predict_word_idx[0,0] = 1
  predicted_sentence = ''

  input_sequence=tokenizer_i.texts_to_sequences([input_sentence])
  inputs=pad_sequences(input_sequence,maxlen=25,padding='post')
  inputs=tf.convert_to_tensor(inputs)

  enc_output, state_h, state_c = model.layers[0](inputs)

  for i in range(DECODER_SEQ_LEN):
      dec_output, state_h, state_c, attention_weights, context_vector = model.layers[1].one_step_decoder(predict_word_idx, enc_output, state_h, state_c)
      predict_word_idx = np.reshape(np.argmax(dec_output), (1, 1))
      predicted_sentence += ' ' + tokenizer_o.index_word[int(predict_word_idx)]

      if tokenizer_o.word_index['<end>'] == predict_word_idx:
          return predicted_sentence
      dec_input = tf.expand_dims([predict_word_idx],0)
  return predicted_sentence